Example subclass of the Graph class.

Copyright (C) 2004-2015 by

Aric Hagberg <hagberg@lanl.gov>

Dan Schult <dschult@colgate.edu>

Pieter Swart <swart@lanl.gov>

All rights reserved.

BSD license.

In [1]:
from networkx import Graph

In [2]:
from networkx.exception import NetworkXException, NetworkXError

In [3]:
import networkx.convert as convert

In [4]:
from copy import deepcopy

In [5]:
class PrintGraph(Graph):
    """
    Example subclass of the Graph class.
    Prints activity log to file or standard output.
    """
    def __init__(self, data=None, name='', file=None, **attr):
        Graph.__init__(self, data=data, name=name, **attr)
        if file is None:
            import sys
            self.fh=sys.stdout
        else:
            self.fh=open(file,'w')

    def add_node(self, n, attr_dict=None, **attr):
        Graph.add_node(self, n, attr_dict=attr_dict, **attr)
        self.fh.write("Add node: %s\n"%n)

    def add_nodes_from(self, nodes, **attr):
        for n in nodes:
            self.add_node(n, **attr)

    def remove_node(self, n):
        Graph.remove_node(self, n)
        self.fh.write("Remove node: %s\n"%n)

    def remove_nodes_from(self, nodes):
        adj = self.adj
        for n in nodes:
            self.remove_node(n)

    def add_edge(self, u, v, attr_dict=None, **attr):  
        Graph.add_edge(self, u, v, attr_dict=attr_dict, **attr)
        self.fh.write("Add edge: %s-%s\n"%(u,v))

    def add_edges_from(self, ebunch, attr_dict=None, **attr):  
        for e in ebunch:
            u,v=e[0:2]
            self.add_edge(u, v, attr_dict=attr_dict, **attr)

    def remove_edge(self, u, v): 
        Graph.remove_edge(self, u, v)
        self.fh.write("Remove edge: %s-%s\n"%(u, v))

    def remove_edges_from(self, ebunch): 
        for e in ebunch:
            u,v=e[0:2]
            self.remove_edge(u, v)

    def clear(self):
        self.name = ''
        self.adj.clear() 
        self.node.clear()
        self.graph.clear()
        self.fh.write("Clear graph\n")

    def subgraph(self, nbunch, copy=True):
        # subgraph is needed here since it can destroy edges in the
        # graph (copy=False) and we want to keep track of all changes.
        #
        # Also for copy=True Graph() uses dictionary assignment for speed
        # Here we use H.add_edge()
        bunch = set(self.nbunch_iter(nbunch))

        if not copy: 
            # remove all nodes (and attached edges) not in nbunch
            self.remove_nodes_from([n for n in self if n not in bunch])
            self.name = "Subgraph of (%s)"%(self.name)
            return self
        else:
            # create new graph and copy subgraph into it       
            H = self.__class__()
            H.name = "Subgraph of (%s)"%(self.name)
            # add nodes
            H.add_nodes_from(bunch)
            # add edges
            seen = set()
            for u, nbrs in self.adjacency():
                if u in bunch:
                    for v, datadict in nbrs.items():
                        if v in bunch and v not in seen:
                            dd = deepcopy(datadict)
                            H.add_edge(u, v, dd)
                    seen.add(u)
            # copy node and graph attr dicts
            H.node = dict( (n, deepcopy(d)) 
                         for (n, d) in self.node.items() if n in H)
            H.graph = deepcopy(self.graph)
            return H

In [6]:
G = PrintGraph()

In [7]:
G.add_node('foo')

Add node: foo


In [8]:
G.add_nodes_from('bar', weight=8)

Add node: b
Add node: a
Add node: r


In [9]:
G.remove_node('b')

Remove node: b


In [10]:
G.remove_nodes_from('ar')

Remove node: a
Remove node: r


In [11]:
list(G.nodes(data=True))

[('foo', {})]

In [12]:
G.add_edge(0, 1, weight=10)

Add edge: 0-1


In [13]:
list(G.edges(data=True))

[(0, 1, {'weight': 10})]

In [14]:
G.remove_edge(0, 1)

Remove edge: 0-1


In [15]:
G.add_edges_from(list(zip(list(range(0o3)), list(range(1, 4)))), weight=10)

Add edge: 0-1
Add edge: 1-2
Add edge: 2-3


In [16]:
list(G.edges(data=True))

[(0, 1, {'weight': 10}), (1, 2, {'weight': 10}), (2, 3, {'weight': 10})]

In [17]:
G.remove_edges_from(list(zip(list(range(0o3)), list(range(1, 4)))))

Remove edge: 0-1
Remove edge: 1-2
Remove edge: 2-3


In [18]:
list(G.edges(data=True))

[]

In [19]:
G = PrintGraph()

In [20]:
G.add_path(list(range(10)))

Add edge: 0-1
Add edge: 1-2
Add edge: 2-3
Add edge: 3-4
Add edge: 4-5
Add edge: 5-6
Add edge: 6-7
Add edge: 7-8
Add edge: 8-9


In [21]:
H1=G.subgraph(list(range(4)), copy=False)

Remove node: 4
Remove node: 5
Remove node: 6
Remove node: 7
Remove node: 8
Remove node: 9


In [22]:
list(H1.edges())

[(0, 1), (1, 2), (2, 3)]